Import labraries

In [ ]:
import faiss
import numpy as np
import pandas as pd
import sklearn as sk
import joblib
import torch

### Loading validation data (with other data this would look different)

In [96]:
index_org = faiss.read_index("validation_it/validation_index.faiss")
index_org.ntotal

3052901

In [3]:
keys = joblib.load("validation_it/rest_keys")

In [ ]:
ids = joblib.load("validation_it/rest__ids")

In [12]:
test_keys = joblib.load("validation_it/validation_keys")

### Loading the clustered data

In [ ]:
new_keys = joblib.load("Validation/validation_results/hdbscan_per_id_it_keys.pkl")
len(new_keys), type(new_keys), new_keys[0]

In [ ]:
new_ids = joblib.load("Validation/validation_results/hdbscan_per_id_it_ids.pkl")
len(new_ids), type(new_ids), new_ids[:5]

In [ ]:
#This is to load data generated by the filter compression https://github.com/jxhe/efficient-knnlm/blob/main/ef_knnlm/dstore_compression/filter_compression.py
new_keys = np.memmap("Validation/Greedy_Merch_paper/Results_with_i/dstore_scorefilter_size201098_embed1280_keys.npy", dtype=np.float16, mode='r', shape=(201098, 1280))
new_ids = np.memmap("Validation/Greedy_Merch_paper/Results_with_i/dstore_scorefilter_size201098_embed1280_vals.npy", dtype=np.int32, mode='r', shape=(201098,1))

### Creating a index
This is done to be able to use a kNN search

In [ ]:
def create_index(index_org, new_keys):
    d = index_org.d
    n_list = index_org.nlist
    n_probe = index_org.nprobe
    m = index_org.pq.M
    nbits = index_org.pq.nbits
    quantizer = faiss.IndexFlatL2(d)
    new_index = faiss.IndexIVFPQ(quantizer, d, n_list, m, nbits)
    new_index.nprobe = n_probe
    new_index.train(new_keys)
    new_index.add(new_keys)
    return new_index

own_index1 = create_index(index_org, new_keys)
own_index1.ntotal, index_org.ntotal

### Comparing the index search results

In [129]:
def get_results_same_place(original_index, new_index, ids, new_ids, search_keys, k):
    _, true_ids_i = original_index.search(search_keys, k)
    _, predicted_ids_i = new_index.search(search_keys, k)
    true_ids = [ids[i] for i in true_ids_i]
    predicted_ids = [new_ids[i] for i in predicted_ids_i]
    matches = sum(1 for i in range(min(len(predicted_ids), len(true_ids))) for j in range(len(predicted_ids[i])) if predicted_ids[i][j] == true_ids[i][j])
    total = sum(len(p) for p in predicted_ids)
    accuracy = (matches / total) * 100 if total > 0 else 0
    return accuracy

In [130]:
get_results_same_place(original_index = index_org, new_index = own_index1, ids = ids, new_ids = new_ids, search_keys = test_keys, k = 1)

96.592

In [131]:
get_results_same_place(original_index = index_org, new_index = own_index1, ids = ids, new_ids = new_ids, search_keys = test_keys, k = 3)

96.23

In [132]:
get_results_same_place(original_index = index_org, new_index = own_index1, ids = ids, new_ids = new_ids, search_keys = test_keys, k = 5)

95.8516

In [133]:
get_results_same_place(original_index = index_org, new_index = own_index1, ids = ids, new_ids = new_ids, search_keys = test_keys, k = 8)

94.953

In [134]:
get_results_same_place(original_index = index_org, new_index = own_index1, ids = ids, new_ids = new_ids, search_keys = test_keys, k = 10)

94.3578

In [135]:
get_results_same_place(original_index = index_org, new_index = own_index1, ids = ids, new_ids = new_ids, search_keys = test_keys, k = 25)

90.56456

Check for how much smaller the new datastore is

In [143]:
100 - own_index1.ntotal / index_org.ntotal * 100

67.0000607165538

In [ ]:
#Exact numbers
index_org.ntotal, own_index1.ntotal

(609389, 201098)